In [1]:
# импортируем необходимые библиотеки, функции и классы
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Модуль_1/Data/Vodafone_missing.csv', sep=';')
data.head(5)

,region,tenure,age,marital,address,income,employ,retire,gender,reside,custcat,churn
0,Region 2,13.0,44.0,mar,9.0,64.0,5.0,no,f,2.0,cat 1,1
1,Region 3,11.0,33.0,mar,7.0,136.0,5.0,no,f,6.0,cat 4,1
2,Region 3,68.0,52.0,mar,24.0,116.0,29.0,no,NaN,NaN,NaN,0
3,Region 2,NaN,33.0,NaN,12.0,NaN,NaN,no,NaN,1.0,cat 1,1
4,Region 2,23.0,30.0,mar,9.0,30.0,2.0,no,f,4.0,cat 3,0


In [3]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data.drop('churn', axis=1), 
                                                    data['churn'], 
                                                    test_size=0.3,
                                                    stratify=data['churn'],
                                                    random_state=42)

In [4]:
# создаем список категориальных переменных, список количественных 
# переменных, не предназначенных для биннинга, список 
# количественных переменных, предназначенных для биннинга 
cat_columns = X_train.dtypes[X_train.dtypes == 'object'].index
num_columns = X_train.dtypes[X_train.dtypes != 'object'].index
num_bin_columns = ['tenure', 'age']

In [5]:
# пишем функцию возведения в квадрат
def power(x):
    x = x ** 2
    return x

In [6]:
# создаем конвейер для количественных переменных,
# которые не будут подвергнуты биннингу
num_pipe = Pipeline([
    ('imp', SimpleImputer()),
    ('yeo_john', PowerTransformer(method='yeo-johnson', standardize=True))
])

# создаем конвейер для количественных переменных,
# которые будут подвергнуты биннингу
num_bin_pipe = Pipeline([
    ('imp', SimpleImputer()),
    ('power', FunctionTransformer(power, validate=False)),
    ('bi', KBinsDiscretizer(encode='onehot-dense'))
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imp', SimpleImputer()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [7]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('num_bin', num_bin_pipe, num_bin_columns),
                ('cat', cat_pipe, cat_columns)]

In [9]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [10]:
# задаем итоговый конвейер
ml_pipe = Pipeline([('tr', transformer), 
                    ('lr', LogisticRegression(solver='lbfgs', max_iter=200))])

In [11]:
# задаем сетку гиперпараметров
param_grid = {
    'tr__num__imp__strategy': ['mean', 'median', 'constant'],
    'tr__num_bin__bi__n_bins': [2, 3],
    'tr__cat__imp__strategy': ['most_frequent', 'constant'],
    'lr__C': [.001, .01, .1]
}

In [12]:
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs = GridSearchCV(ml_pipe, param_grid, cv=5, return_train_score=False)
# выполняем решетчатый поиск
gs.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(
    gs.best_params_))
# смотрим наилучшее значение правильности
print('Наилучшее значение правильности: {:.3f}'.format(
    gs.best_score_))
# смотрим значение правильности на тестовой выборке
print('Значение правильности на тестовой выборке: {:.3f}'.format(
    gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'lr__C': 0.1, 'tr__cat__imp__strategy': 'most_frequent', 'tr__num__imp__strategy': 'constant', 'tr__num_bin__bi__n_bins': 3}
Наилучшее значение правильности: 0.763
Значение правильности на тестовой выборке: 0.760


In [13]:
# запишем результаты перекрестной 
# проверки в DataFrame
results = pd.DataFrame(gs.cv_results_)
# превращаем в сводную таблицу
table = results.pivot_table(values=['mean_test_score'],    
                            index=['param_lr__C',
                                   'param_tr__cat__imp__strategy',
                                   'param_tr__num__imp__strategy',
                                   'param_tr__num_bin__bi__n_bins'])
# сортируем по убыванию правильности
table = table.sort_values(by='mean_test_score', ascending=False)
print(table)

mean_test_score
param_lr__C param_tr__cat__imp__strategy param_tr__num__imp__strategy param_tr__num_bin__bi__n_bins                 
0.100       most_frequent                constant                     3                                     0.762857
            constant                     constant                     3                                     0.760000
            most_frequent                constant                     2                                     0.757143
            constant                     constant                     2                                     0.755714
            most_frequent                median                       3                                     0.754286
            constant                     median                       2                                     0.754286
                                         mean                         2                                     0.752857
                                                